## Import libraries

In [26]:
import numpy as np
import pandas as pd

## Define Functions

In [27]:
def read_dataset(feature_file, label_file):
    ''' Read data set in *.csv to data frame in Pandas'''
    df_X = pd.read_csv(feature_file)
    df_y = pd.read_csv(label_file)
    X = df_X.values  # convert values in dataframe to numpy array (features)
    y = df_y.values  # convert values in dataframe to numpy array (label)
    return X, y

def RMSE(ypred, yexact):
    return np.sqrt(np.sum((ypred - yexact)**2)/ ypred.shape[0])

def PCC(y_pred, y_test):
    from scipy import stats
    a = y_test
    b = y_pred
    accuracy = stats.pearsonr(a, b)
    return accuracy

In [28]:
class decisiontrees():
    def __init__(self, max_depth=5, current_depth=1,mode = 'Classification'):        
        ''' initalize the decision trees parameters '''
        self.max_depth = max_depth
        self.current_depth = current_depth
        self.left_tree = None
        self.right_tree = None
        self.mode = mode
        
    def fit(self, X, y):
        self.X = X
        self.y = y        
        self.n_features = X.shape[1]
        self.n_samples = X.shape[0]
        if self.current_depth <= self.max_depth:
#             print('Current depth = %d' % self.current_depth)
            if self.mode == 'Classification':
                self.GINI = self.GINI_calculation(self.y)
                self.best_feature_id, self.best_gain, self.best_split_value = self.find_best_split()
                if self.best_gain > 0:
                    self.split_trees()
            if self.mode == 'Regression': # Homework
                self.SD = np.std(self.y)
                self.best_feature_id, self.best_score, self.best_split_value = self.find_best_split()
                if self.SD > 0:
                    self.split_trees()
        
    def predict(self, X_test): # predict all the samples in the test set by tree propagation for each sample
        n_test = X_test.shape[0]
        ypred = np.zeros(n_test, dtype=int)  
        for i in range(n_test):
            ypred[i] = self.tree_propogation(X_test[i])            
        return ypred
                
    def tree_propogation(self, feature): # goes through the tree to predict one sample
        if self.is_leaf_node(): # if the node is a leaf, we predict the label
            return self.predict_label()
        if feature[self.best_feature_id] < self.best_split_value:  # choose which path to take based on best split value
            child_tree = self.left_tree
        else:
            child_tree = self.right_tree
        return child_tree.tree_propogation(feature) # recursively call the tree propagation
    
    def predict_label(self): # predicts the label in a leaf node
        if self.mode == 'Classification':
            unique, counts = np.unique(self.y, return_counts=True) # find the labels in the leaf node and their counts
            label = None  # initialize label and largest count
            max_count = 0
            for i in range(unique.size):  # go through the counts of each label
                if counts[i] > max_count: # find the label that has the most samples associated with it
                    max_count = counts[i]
                    label = unique[i]
            return label
        if self.mode == 'Regression':
            label = np.average(self.y)
            return label
    
    def is_leaf_node(self):
        return self.left_tree is None
    
    def split_trees(self):
        # create a left tree
        self.left_tree = decisiontrees(max_depth = self.max_depth, current_depth = self.current_depth + 1, mode = self.mode)
        self.right_tree = decisiontrees(max_depth = self.max_depth, current_depth = self.current_depth + 1, mode = self.mode)
        best_feature_values = self.X[:, self.best_feature_id]
        left_indices = np.where(best_feature_values < self.best_split_value)
        right_indices = np.where(best_feature_values >= self.best_split_value)
        left_tree_X = self.X[left_indices]
        left_tree_y = self.y[left_indices]
        right_tree_X = self.X[right_indices]
        right_tree_y = self.y[right_indices]
        
        # fit left and right tree
        self.left_tree.fit(left_tree_X, left_tree_y)
        self.right_tree.fit(right_tree_X, right_tree_y)
        
    
    def find_best_split(self):
        best_feature_id = None
        best_gain = 0
        best_split_value = None
        for feature_id in range(self.n_features):
            current_gain, current_split_value = self.find_best_split_one_feature(feature_id)
            if current_gain is None:
                continue
            if best_gain < current_gain:
                best_feature_id = feature_id
                best_gain = current_gain                
                best_split_value = current_split_value
        return best_feature_id, best_gain, best_split_value
    
    def find_best_split_one_feature(self, feature_id):
        '''
            Return information_gain, split_value
        '''
        feature_values = self.X[:, feature_id]
        unique_feature_values = np.unique(feature_values)
        best_gain = 0.0
        best_split_value = None
        if len(unique_feature_values) == 1:
            return best_gain, best_split_value
        for fea_val in unique_feature_values:
            left_indices = np.where(feature_values < fea_val)
            right_indices = np.where(feature_values >= fea_val)
            left_tree_X = self.X[left_indices]
            left_tree_y = self.y[left_indices]

            right_tree_X = self.X[right_indices]
            right_tree_y = self.y[right_indices]
            left_GINI = self.GINI_calculation(left_tree_y)
            right_GINI = self.GINI_calculation(right_tree_y)
            left_score = np.std(left_tree_y)
            right_score = np.std(right_tree_y)

            # calculate gain
            if self.mode == 'Classification':
                left_n_samples = left_tree_X.shape[0]
                right_n_samples = right_tree_X.shape[0]
                current_gain = self.GINI - (left_n_samples/self.n_samples * left_GINI + right_n_samples/self.n_samples * right_GINI)
            if self.mode == 'Regression':
                left_n_samples = left_tree_X.shape[0]
                right_n_samples = right_tree_X.shape[0]
                current_gain = self.SD - (left_n_samples/self.n_samples * left_score + right_n_samples/self.n_samples * right_score)
            if best_gain < current_gain:
                best_gain = current_gain
                best_split_value = fea_val
        return best_gain, best_split_value                                    
            
    def GINI_calculation(self, y):
        if y.size == 0 or y is None:
            return 0.0
        unique, counts = np.unique(y, return_counts=True)
        prob = counts/y.size
        return 1.0 - np.sum(prob*prob)

In [30]:
# ===================================Regression==================================
# Homework
X_train, y_train = read_dataset('airfoil_self_noise_X_train.csv', 'airfoil_self_noise_y_train.csv')
X_test, y_test = read_dataset('airfoil_self_noise_X_test.csv', 'airfoil_self_noise_y_test.csv')
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
mydt = decisiontrees(max_depth=11, mode='Regression')
mydt.fit(X_train, y_train)
y_pred = mydt.predict(X_test)
print('PCC of our model ', PCC(y_pred, y_test.ravel()))
print('RSME of our model ', RMSE(y_pred, y_test.ravel()))

(1202, 5)
(301, 5)
(1202, 1)
(301, 1)
PCC of our model  (0.9110566947980279, 4.4333587043288576e-117)
RSME of our model  2.829249949323586
